# Данные

In [56]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

X, y = make_regression(n_features=100, n_samples=1000)
model_sk =  LinearRegression( )
model_sk.fit(X, y)
w = model_sk.coef_

X.shape, y.shape, w.shape

((1000, 100), (1000,), (100,))

# Собираем корпус под модель


In [57]:
from sklearn.base import RegressorMixin, BaseEstimator
import numpy as np

class AwesomeRegression(RegressorMixin, BaseEstimator):
    
    def __init__(self, tol=1e-6, lr=0.001):
        self.w = None
        self.tol = tol
        self.lr = lr
        self.iter = 0
        
    def mse_loss(self, X, y):
        return ((y - X@self.w).T)@(y - X@self.w)/y.size

    def grad_loss(self, X, y):
        return -2*X.T@(y - X@self.w)/y.size
           
    def fit(self, X, y):
        self.w = np.random.normal(size=(X.shape[1]))
        
        new_w = self.w - self.lr*self.grad_loss(X, y)
        
        while np.any(abs(new_w - self.w) > self.tol):
            self.iter += 1
            self.w = new_w
            new_w = self.w - self.lr*self.grad_loss(X, y)

    def fit_formula(X, y):
        self.w = np.linalg.inv(X.T@X)@(X.T@y)
     
    def predict(self, X):
        return X@self.w

In [58]:
model = AwesomeRegression()
model.fit(X, y)

model.iter

9454

In [65]:
np.linalg.norm(model_sk.coef_ - model.w, ord=2)

0.0028147674322374604

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
from scipy import special

from sklearn.datasets import make_classification
from numpy.linalg import norm

# Используйте scipy.special для вычисления численно неустойчивых функций
# https://docs.scipy.org/doc/scipy/reference/special.html#module-scipy.special

def lossf(w, X, y, l1, l2):
    """
    Вычисление функции потерь.

    :param w: numpy.array размера  (M,) dtype = np.float
    :param X: numpy.array размера  (N, M), dtype = np.float
    :param y: numpy.array размера  (N,), dtype = np.int
    :param l1: float, l1 коэффициент регуляризатора 
    :param l2: float, l2 коэффициент регуляризатора 
    :return: float, value of loss function
    """
    
    lossf = np.sum(np.log(1+np.exp(-y*np.dot(X,w)))) + l1*norm(w, ord=1) + l2* (norm(w,ord=2))**2
    return lossf

def gradf(w, X, y, l1, l2):
    """
    Вычисление градиента функции потерь.

    :param w: numpy.array размера  (M,), dtype = np.float
    :param X: numpy.array размера  (N, M), dtype = np.float
    :param y: numpy.array размера  (N,), dtype = np.int
    :param l1: float, l1 коэффициент регуляризатора 
    :param l2: float, l2 коэффициент регуляризатора 
    :return: numpy.array размера  (M,), dtype = np.float, gradient vector d lossf / dw
    """
    chisl = -y*np.exp(-y*(np.dot(X,w)))
    znam = 1+ np.exp(-y*(np.dot(X,w)))
    const = chisl/znam
    gradw = np.sum((const*X.T),axis=1) + 2*l2*w + l1*np.sign(w)
    return gradw


class LR(ClassifierMixin, BaseEstimator):
    def __init__(self, lr=1, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0):
        self.l1 = l1
        self.l2 = l2
        self.w = None
        self.lr = lr
        self.verbose = verbose
        self.num_iter = num_iter
        
        
    def fit(self, X, y):
        """
        Обучение логистической регрессии.
        Настраивает self.w коэффициенты модели.

        Если self.verbose == True, то выводите значение 
        функции потерь на итерациях метода оптимизации. 

        :param X: numpy.array размера  (N, M), dtype = np.float
        :param y: numpy.array размера  (N,), dtype = np.int
        :return: self
        """
        n, d = X.shape
        self.w = np.random.normal(size=d)
        
        t = 0
        err = [lossf(self.w, X, y, self.l1, self.l2)]     
                
        while t < self.num_iter:
            t += 1
            dw = gradf(self.w, X, y, self.l1, self.l2)
            
            #self.w -= dw * self.lr
            while lossf(self.w-dw*self.lr, X, y, self.l1, self.l2) > \
                           lossf(self.w, X, y, self.l1, self.l2) - 0.1*self.lr*norm(dw,ord=2)**2:
                self.lr = 0.95*self.lr
            self.w -= dw * self.lr                        
            
            err.append(lossf(self.w, X, y, self.l1, self.l2))
            if self.verbose == 1:
                print(err[t])      
        return self.w
  
    
    def predict_proba(self, X):
        """
        Предсказание вероятности принадлежности объекта к классу 1.
        Возвращает np.array размера (N,) чисел в отрезке от 0 до 1.

        :param X: numpy.array размера  (N, M), dtype = np.float
        :return: numpy.array размера  (N,), dtype = np.int
        """
        # Вычислите вероятности принадлежности каждого 
        # объекта из X к положительному классу, используйте
        # эту функцию для реализации LR.predict
        probs =1/(1+np.exp(-np.dot(X,self.w)))
        return probs
     
    def predict(self, X):
        """
        Предсказание класса для объекта.
        Возвращает np.array размера (N,) элементов 1 или -1.

        :param X: numpy.array размера  (N, M), dtype = np.float
        :return:  numpy.array размера  (N,), dtype = np.int
        """
        # Вычислите предсказания для каждого объекта из X
        predicts = self.predict_proba(X)
        predicts[predicts >= 0.5] = 1
        predicts[predicts < 0.5] = -1 
        return predicts 


def test_work():
    print "Start test"
    X, y = make_classification(n_features=100, n_samples=1000)
    y = 2 * (y - 0.5)

    try:
        clf = LR(lr=0.01, l1=1e-4, l2=1e-4, num_iter=1000, verbose=0)
    except Exception:
        assert False, "Создание модели завершается с ошибкой"
        return

    try:
        clf.fit(X, y)
    except Exception:
        assert False, "Обучение модели завершается с ошибкой"
        return

    assert isinstance(lossf(clf.w, X, y, 1e-3, 1e-3), float), "Функция потерь должна быть скалярной и иметь тип np.float"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).shape == (100,), "Размерность градиента должна совпадать с числом параметров"
    assert gradf(clf.w, X, y, 1e-3, 1e-3).dtype == np.float, "Вектор градиента, должен состоять из элементов типа np.float"
    assert clf.predict(X).shape == (1000,), "Размер вектора предсказаний, должен совпадать с количеством объектов"
    assert np.min(clf.predict_proba(X)) >= 0, "Вероятности должны быть не меньше, чем 0"
    assert np.max(clf.predict_proba(X)) <= 1, "Вероятности должны быть не больше, чем 1"
    assert len(set(clf.predict(X))) == 2, "Метод предсказывает больше чем 2 класса на двух классовой задаче"
    print "End tests"

test_work()